In [ ]:
!pip install tensorflow-gpu==2.1.0

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import time
import os

#using tensorflow keras as backend to build neural nets
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras.datasets import cifar10
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report   #confusion matrix to find the accuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger

from PIL import Image

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [4]:
!unzip '/content/gdrive/My Drive/Cifar10Assignment.zip'

Archive:  /content/gdrive/My Drive/Cifar10Assignment.zip
  inflating: Cifar10TrainTensoflowModel.ipynb  
  inflating: modelcheckpoint.hdf5    
  inflating: saved_model.pb          
  inflating: TensorFlow_with_GPU.ipynb  
  inflating: variables.data-00000-of-00002  
  inflating: variables.data-00001-of-00002  
  inflating: variables.index         
  inflating: cifar10_cnnWeightModel.h5  
  inflating: cifar10_results.csv     


In [5]:

model_load_tf = tf.keras.models.load_model('cifar10_cnnWeightModel.h5')
model_load_tf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

#Convert Keras model into TensorRT model

In [6]:
#insatall TensorRT 6.x
!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/libnvinfer6_6.0.1-1+cuda10.1_amd64.deb
!dpkg -i libnvinfer6_6.0.1-1+cuda10.1_amd64.deb

--2020-07-01 13:46:28--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/libnvinfer6_6.0.1-1+cuda10.1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71245796 (68M) [application/x-deb]
Saving to: ‘libnvinfer6_6.0.1-1+cuda10.1_amd64.deb’

libnvinfer6_6.0.1-1 100%[===================>]  67.94M   166MB/s    in 0.4s    

2020-07-01 13:46:28 (166 MB/s) - ‘libnvinfer6_6.0.1-1+cuda10.1_amd64.deb’ saved [71245796/71245796]

Selecting previously unselected package libnvinfer6.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack libnvinfer6_6.0.1-1+cuda10.1_amd64.deb ...
Unpacking libnvinfer6 (6.0.1-1+cuda10.1) ...
Setting up libnvinfer6 (6.0.1-1+cuda10.1) ...
Processing triggers for libc-bin (2.27-3ubun

#link TensorRT with Tensorflow

In [2]:
from tensorflow.compiler.tf2tensorrt.wrap_py_utils import get_linked_tensorrt_version
from tensorflow.compiler.tf2tensorrt.wrap_py_utils import get_loaded_tensorrt_version
print(f"Linked TensorRT version {get_linked_tensorrt_version()}")
print(f"Loaded TensorRT version {get_loaded_tensorrt_version()}")

Linked TensorRT version (6, 0, 1)
Loaded TensorRT version (6, 0, 1)


In [4]:
# # force reset ipython namespaces
# %reset -f
#Convert .h5 model to .pb graph model
tf.compat.v1.disable_eager_execution()
# Clear any previous session.
tf.keras.backend.clear_session()
import tensorflow as tf
print(tf.__version__)

2.1.0


#Model conersion .h5 format to .pb graph format

In [7]:
save_pb_dir = ''
model_fname = 'cifar10_cnnWeightModel.h5'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.compat.v1.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen =  tf.compat.v1.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

# This line must be executed before loading Keras model.
tf.keras.backend.set_learning_phase(0) 

model = tf.keras.models.load_model(model_fname)

session = tf.compat.v1.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

['conv2d_input_1'] ['dense_1_1/Softmax']
INFO:tensorflow:Froze 40 variables.
INFO:tensorflow:Converted 40 variables to const ops.


In [15]:
!nvidia-smi

Wed Jul  1 10:01:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    57W / 149W |    296MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash
wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

dpkg -i nvidia-machine-learning-repo-*.deb
apt-get update

In [ ]:
# #Download TensorRT. 5.1.2.2
# !apt-get install libnvinfer5
# !wget -O nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb https://www.dropbox.com/s/45pz13r4e8ip4bl/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb?dl=0
# !dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb
# !apt-key add /var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227/7fa2af80.pub
!apt-get update
# !apt-get install -y --no-install-recommends libnvinfer5=5.1.2-1+cuda10.0
# !apt-get install -y --no-install-recommends libnvinfer-dev=5.1.2-1+cuda10.0
!apt-get install tensorrt
!apt-get install python3-libnvinfer-dev

In [20]:
from tensorflow.compiler.tf2tensorrt.wrap_py_utils import get_linked_tensorrt_version
from tensorflow.compiler.tf2tensorrt.wrap_py_utils import get_loaded_tensorrt_version
print(f"Linked TensorRT version {get_linked_tensorrt_version()}")
print(f"Loaded TensorRT version {get_loaded_tensorrt_version()}")

Linked TensorRT version (6, 0, 1)
Loaded TensorRT version (6, 0, 1)


In [21]:
!dpkg -l | grep TensorRT

ii  libnvinfer-dev                          7.1.3-1+cuda11.0                                  amd64        TensorRT development libraries and headers
ii  libnvinfer-plugin-dev                   7.1.3-1+cuda11.0                                  amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7                      7.1.3-1+cuda11.0                                  amd64        TensorRT plugin libraries
ii  libnvinfer6                             6.0.1-1+cuda10.1                                  amd64        TensorRT runtime libraries
ii  libnvinfer7                             7.1.3-1+cuda11.0                                  amd64        TensorRT runtime libraries
ii  libnvonnxparsers-dev                    7.1.3-1+cuda11.0                                  amd64        TensorRT ONNX libraries
ii  libnvonnxparsers7                       7.1.3-1+cuda11.0                                  amd64        TensorRT ONNX libraries
ii  libnvparsers-dev                        7.1.3-1+cu

In [22]:
!dpkg -l | grep nvinfer

ii  libnvinfer-dev                          7.1.3-1+cuda11.0                                  amd64        TensorRT development libraries and headers
ii  libnvinfer-plugin-dev                   7.1.3-1+cuda11.0                                  amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7                      7.1.3-1+cuda11.0                                  amd64        TensorRT plugin libraries
ii  libnvinfer6                             6.0.1-1+cuda10.1                                  amd64        TensorRT runtime libraries
ii  libnvinfer7                             7.1.3-1+cuda11.0                                  amd64        TensorRT runtime libraries
ii  python3-libnvinfer                      7.1.3-1+cuda11.0                                  amd64        Python 3 bindings for TensorRT
ii  python3-libnvinfer-dev                  7.1.3-1+cuda11.0                                  amd64        Python 3 development package for TensorRT


#Check if Tensor Core GPU is Present with compute_capability ==6.0 (TensorRT version)

In [8]:
from tensorflow.python.client import device_lib

def check_tensor_core_gpu_present():
    local_device_protos = device_lib.list_local_devices()
    for line in local_device_protos:
        if "compute capability" in str(line):
            compute_capability = float(line.physical_device_desc.split("compute capability: ")[-1])
            if compute_capability>=6.0:
                return True
    
print("Tensor Core GPU Present:", check_tensor_core_gpu_present())
tensor_core_gpu = check_tensor_core_gpu_present()

Tensor Core GPU Present: True


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


### Inference with native TF2.0 saved model


In [10]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 13s 0us/step


In [11]:
X_test = X_test.astype('float32')
X_test = X_test / 255.0

In [13]:
%%time
model_fname = 'cifar10_cnnWeightModel.h5'
LoadNaivemodel = tf.keras.models.load_model(model_fname)
LoadNaivemodel.predict(X_test)


CPU times: user 4.63 s, sys: 951 ms, total: 5.58 s
Wall time: 9.64 s


### TF-TRT FP32 model

We first convert the TF native FP32 model to a TF-TRT FP32 model.

In [14]:

from tensorflow.python.compiler.tensorrt import trt_convert as trt
print('Converting to TF-TRT FP32...')
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=8000000000, #1 << 25,
    precision_mode='FP32',
    minimum_segment_size=50,
    is_dynamic_op=True
)
graph_io.write_graph(trt_graph, ".","cifar10NetFP32_trt_graph_saved_model.pb", as_text=False)
print('Done Converting to TF-TRT FP32')

Converting to TF-TRT FP32...
INFO:tensorflow:Linked TensorRT version: (6, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (6, 0, 1)
Done Converting to TF-TRT FP32


### TF-TRT FP16 model

In [15]:

from tensorflow.python.compiler.tensorrt import trt_convert as trt
print('Converting to TF-TRT FP16...')
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=8000000000, #1 << 25,
    precision_mode='FP16',
    minimum_segment_size=80,
    is_dynamic_op=True
)
graph_io.write_graph(trt_graph, ".","cifar10NetFP16_trt_graph_saved_model.pb", as_text=False)
print('Done Converting to TF-TRT FP16')

Converting to TF-TRT FP16...
INFO:tensorflow:Linked TensorRT version: (6, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (6, 0, 1)
Done Converting to TF-TRT FP16


### TF-TRT INT8 model

In [16]:

from tensorflow.python.compiler.tensorrt import trt_convert as trt
print('Converting to TF-TRT INT8...')
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=8000000000, #1 << 25,
    precision_mode='INT8',
    minimum_segment_size=50,
    is_dynamic_op=True
)
graph_io.write_graph(trt_graph, ".","cifar10NetINT8_trt_graph_saved_model.pb", as_text=False)
print('Done Converting to TF-TRT INT8')

Converting to TF-TRT INT8...
INFO:tensorflow:Linked TensorRT version: (6, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (6, 0, 1)
Done Converting to TF-TRT INT8
